# Подключения и функции

In [187]:
# Импорт

import pandas as pd
from clickhouse_driver import Client
import warnings
warnings.filterwarnings('ignore')

In [188]:
# Функция создания датафрейма

def create_dataframe(user, password, table_name='films'):
    client = Client('90.156.216.60', user=user, password=password, database='it_minimalist_CH')
    query = f'SELECT * FROM {table_name}'
    df = client.query_dataframe(query)
    return df

В ячейке ниже создается датафрейм в переменной `df`.
<br>

- Замените `'your_username'`, `'your_password'` на ваши учетные данные, которые получили на почту (у каждого своя учетная запись с индивидуальным доступом к БД Clickhouse).
<br>
- Как вы заметили в параметрах функции `create_dataframe`, по умолчанию датафрейм создатеся из таблицы `'films'`. Также в базе данных лежит таблица `'payments'`

In [189]:
films = create_dataframe(user='tanya25pm', password='klcemshu', table_name='films')
payments = create_dataframe(user='tanya25pm', password='klcemshu', table_name='payments')

In [190]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"


# Семинар 4

Начнем с практики (*вне контекста пандаса*)

In [110]:
def is_cheap(price):
    if price <= 200:
        return 'cheap'
    else:
        return 'expensive'

In [111]:
is_cheap(210)

'expensive'

In [57]:
price = 300
price <= 200

False

In [58]:
# ТАК НЕ НАДО!!!!!!!
def is_cheap(price):
    if price <= 200:
        return True
    else:
        return False

In [59]:
is_cheap(210)

False

In [112]:
# PRO-вариант
def is_cheap(price):
    return price <= 200

In [113]:
is_cheap(210)

False

In [62]:
is_cheap(190)

True

## Лямбда-вариант

In [114]:
if_exp = lambda price: 'cheap' if price <=200 else 'expensive'

In [64]:
if_exp(210)

'expensive'

In [65]:
if_exp(190)

'cheap'

- Лямбда всегда содержит ТОЛЬКО одно выражение, имеет ограниченный спектр применения,
- Нет `return`, само определение функции содержит возвращенное выражение,
- Не нужно присваивать его переменной (но можно),
- Применяется когда в моменте нужна функция одноразово, и нет смысла писать полноценную функцию,
- Часто используется, как аргумент функции высшего порядка (об этом будем говорить отдельно, скоро на основном канале продолжится цикл уроков, там будеи разбирать продвинутый python), так вот лямбды используются, как функция, которая принимает другие функции в качестве аргументов (часто — filter(), map(),reduce(), разберем их.)

### Переходим к пандасу

Когда нам нужно применить какой-то эдакий алгоритм, любой к столбцу пандаса, к *series*, мы используем для этого метод `APPLY`<br>
Принимает на вход **Функцию**, применяется к **Series**

ОТЛИЧАЕТСЯ ТЕМ, ЧТО ПРИМЕНЯЕТСЯ ПОСТРОЧНО (*т.е. к тем объектам, котоыре лежат в ячейках этого столбца (этого series), к которому применяется*

### Пример 1

#### .apply

In [66]:
def is_cheap(price):
    if price <= 200:
        return 'cheap'
    else:
        return 'expensive'

In [67]:
is_cheap(190)

'cheap'

In [68]:
films['is_cheap'] = films.price_ticket.apply(is_cheap)

In [69]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive


In [70]:
films = films.assign(is_cheap_2 = films.price_ticket.apply(lambda price: 'cheap' if price <=200 else 'expensive'))
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap,is_cheap_2
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap,cheap
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap,cheap
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive,expensive


### Пример 2

Хотим сделать все названия фильмов прописными

In [115]:
films.film_name = films.film_name.apply(lambda film_name: film_name.lower())

### Пример 3

ДЛЯ АФИШИ

In [116]:
films['name_length_for_poster'] = films.film_name.apply(lambda film_name: len(film_name))

In [117]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13


**ВАЖНО!**

НЕ ИСПОЛЬЗОВАТЬ `apply`, когда можно использовать стандартные методы, способы манипуляции над пандами

## .str (аксессор)
https://pandas.pydata.org/docs/user_guide/text.html

In [118]:
films_short = films.head(8)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18


In [119]:
# Определите, начинается ли каждая строка с соответствия регулярному выражению.
films_short['film_name'].str.match('Кр')

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
Name: film_name, dtype: bool

In [120]:
# Считается кол-во подстрок. Например кол-во пробелов
films_short['Кол-во пробелов'] = films_short['film_name'].str.count(' ')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17,2
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13,1
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15,2
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22,2
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15,1
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18,1


In [121]:
films_short['film_name'].str.lower()

0                      побег из шоушенка
1                          крёстный отец
2                          тёмный рыцарь
3                        крёстный отец 2
4                 12 разгневанных мужчин
5                        список шиндлера
6    властелин колец: возвращение короля
7                     криминальное чтиво
Name: film_name, dtype: object

In [122]:
films_short['film_name'].str.upper()

0                      ПОБЕГ ИЗ ШОУШЕНКА
1                          КРЁСТНЫЙ ОТЕЦ
2                          ТЁМНЫЙ РЫЦАРЬ
3                        КРЁСТНЫЙ ОТЕЦ 2
4                 12 РАЗГНЕВАННЫХ МУЖЧИН
5                        СПИСОК ШИНДЛЕРА
6    ВЛАСТЕЛИН КОЛЕЦ: ВОЗВРАЩЕНИЕ КОРОЛЯ
7                     КРИМИНАЛЬНОЕ ЧТИВО
Name: film_name, dtype: object

In [123]:
films_short['film_name'].str.len()

0    17
1    13
2    13
3    15
4    22
5    15
6    35
7    18
Name: film_name, dtype: int64

In [124]:
films_short['film_name_mistake'] = films_short['film_name'].apply(lambda x: '  ' + x + ' ')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов,film_name_mistake
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17,2,побег из шоушенка
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13,1,крёстный отец
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1,тёмный рыцарь
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15,2,крёстный отец 2
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22,2,12 разгневанных мужчин
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15,1,список шиндлера
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3,властелин колец: возвращение короля
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18,1,криминальное чтиво


In [125]:
films_short['film_name_mistake'].to_list()

['  побег из шоушенка ',
 '  крёстный отец ',
 '  тёмный рыцарь ',
 '  крёстный отец 2 ',
 '  12 разгневанных мужчин ',
 '  список шиндлера ',
 '  властелин колец: возвращение короля ',
 '  криминальное чтиво ']

In [126]:
# strip убрать пробелы в начале и в конце
films_short['film_name_correct_all'] = films_short['film_name_mistake'].str.strip()

In [127]:
films_short['film_name_correct_all'].to_list()

['побег из шоушенка',
 'крёстный отец',
 'тёмный рыцарь',
 'крёстный отец 2',
 '12 разгневанных мужчин',
 'список шиндлера',
 'властелин колец: возвращение короля',
 'криминальное чтиво']

In [128]:
# lstrip убрать пробелы в начале (слева left)
films_short['film_name_correct_left'] = films_short['film_name_mistake'].str.lstrip()

In [129]:
films_short['film_name_correct_left'].to_list()

['побег из шоушенка ',
 'крёстный отец ',
 'тёмный рыцарь ',
 'крёстный отец 2 ',
 '12 разгневанных мужчин ',
 'список шиндлера ',
 'властелин колец: возвращение короля ',
 'криминальное чтиво ']

In [130]:
# rstrip убрать пробелы в конце (справа right)
films_short['film_name_correct_right'] = films_short['film_name_mistake'].str.rstrip()

In [131]:
films_short['film_name_correct_right'].to_list()

['  побег из шоушенка',
 '  крёстный отец',
 '  тёмный рыцарь',
 '  крёстный отец 2',
 '  12 разгневанных мужчин',
 '  список шиндлера',
 '  властелин колец: возвращение короля',
 '  криминальное чтиво']

In [132]:
films_short[['kind_film']]

,kind_film
0,драма
1,"детектив, драма"
2,"боевик, детектив, драма"
3,"детектив, драма"
4,"детектив, драма"
5,"байопик, драма, исторический фильм"
6,"боевик, приключение, драма"
7,"детектив, драма"


In [133]:
df_kind_film = films_short['kind_film'].str.split(', ', expand=True)
df_kind_film

,0,1,2
0,драма,None,None
1,детектив,драма,None
2,боевик,детектив,драма
3,детектив,драма,None
4,детектив,драма,None
5,байопик,драма,исторический фильм
6,боевик,приключение,драма
7,детектив,драма,None


In [134]:
df_kind_film.columns = ['Жанр 1', 'Жанр 2', 'Жанр 3']
df_kind_film

,Жанр 1,Жанр 2,Жанр 3
0,драма,None,None
1,детектив,драма,None
2,боевик,детектив,драма
3,детектив,драма,None
4,детектив,драма,None
5,байопик,драма,исторический фильм
6,боевик,приключение,драма
7,детектив,драма,None


In [135]:
df_kind_film['Жанр 1'].str.cat(sep=' | ')

'драма | детектив | боевик | детектив | детектив | байопик | боевик | детектив'

## np.where
<div>
<img src="attachment:021395c0-4d19-4ca6-a17a-956a7b61723d.png" width="500"/>
</div>

In [136]:
import numpy as np

In [93]:
films_short = films.head(8)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap,is_cheap_2,name_length_for_poster
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap,cheap,17
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap,cheap,13
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive,expensive,13
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",expensive,expensive,15
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",expensive,expensive,22
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",cheap,cheap,15
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",expensive,expensive,35
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",expensive,expensive,18


In [137]:
films_short['is_exp'] = np.where(films_short['price_ticket'] >= 200, 'expensive', 'cheap')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов,film_name_mistake,film_name_correct_all,film_name_correct_left,film_name_correct_right,is_exp
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17,2,побег из шоушенка,побег из шоушенка,побег из шоушенка,побег из шоушенка,cheap
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13,1,крёстный отец,крёстный отец,крёстный отец,крёстный отец,cheap
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,expensive
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15,2,крёстный отец 2,крёстный отец 2,крёстный отец 2,крёстный отец 2,expensive
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22,2,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,expensive
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15,1,список шиндлера,список шиндлера,список шиндлера,список шиндлера,cheap
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,expensive
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18,1,криминальное чтиво,криминальное чтиво,криминальное чтиво,криминальное чтиво,expensive


In [138]:
films_short[
(films_short['price_ticket'] >= 200) & 
(films_short['release_year'] >= 2000)
]

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов,film_name_mistake,film_name_correct_all,film_name_correct_left,film_name_correct_right,is_exp
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,expensive
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,expensive


In [139]:
films_short['сложное условие'] = np.where(
    (films_short['price_ticket'] >= 200) & (films_short['release_year'] >= 2000), 
    1, 
    0
)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов,film_name_mistake,film_name_correct_all,film_name_correct_left,film_name_correct_right,is_exp,сложное условие
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17,2,побег из шоушенка,побег из шоушенка,побег из шоушенка,побег из шоушенка,cheap,0
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13,1,крёстный отец,крёстный отец,крёстный отец,крёстный отец,cheap,0
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,expensive,1
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15,2,крёстный отец 2,крёстный отец 2,крёстный отец 2,крёстный отец 2,expensive,0
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22,2,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,expensive,0
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15,1,список шиндлера,список шиндлера,список шиндлера,список шиндлера,cheap,0
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,expensive,1
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18,1,криминальное чтиво,криминальное чтиво,криминальное чтиво,криминальное чтиво,expensive,0


In [140]:
films_short['сложное условие'].mean()

0.25

In [141]:
print('Доля таких фильмов: ' + str(films_short['сложное условие'].mean()*100) + '%')

Доля таких фильмов: 25.0%


# Домашнее задание по семинару 👨‍💻

## 1. Напишите несколько Лямбда-функций: 
1.1 которая принимаюет строку и возвращает ее длину <br>
1.2 которая принимает число и возвращает его квадрат<br>
1.3 которая принимает `float` и возвращает его мантиссу (все, что после плавающей точки). *Тип данных вывода не важен.*

In [99]:
# your code 
# 1.1
len_string = lambda string: len(string)
print(len_string('Бухгалтерия'))

11


In [100]:
#1.2
sq_num = lambda num: num**2
print(sq_num(43))

1849


In [101]:
#1.3 
import math
mant_num = lambda x: math.fmod(x, 1)
print(mant_num(10.11111))

0.11111000000000004


## 2. Проверка требований к афише

На основании созданного на семинаре столбца `name_length_for_poster` (для этого надо прогнать весь код выше), создайте столбец `is_fit_to_poster`, в котором будет результат, поместится ли название на афишу (помещается название до 15 символов) <br><br>
Сделайте двумя способами (с помощью обычного логического выражения и с помощью связки **apply + лямбда-функция**)

In [166]:
# your code 
# **apply + лямбда-функция**
films_short['is_fit_to_poster'] = films_short['film_name'].apply(lambda name: 'поместится' if len(name) < 15 else 'не поместится')


In [167]:
# используем логическое выражение

films_short['is_fit_to_poster_2'] = np.where(
    films_short['film_name'].str.len() < 15,
    'влезет', 
    'не влезет'
)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,name_length_for_poster,Кол-во пробелов,film_name_mistake,film_name_correct_all,film_name_correct_left,film_name_correct_right,is_exp,сложное условие,is_fit_to_poster,is_fit_to_poster_2
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,17,2,побег из шоушенка,побег из шоушенка,побег из шоушенка,побег из шоушенка,cheap,0,не поместится,не влезет
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",13,1,крёстный отец,крёстный отец,крёстный отец,крёстный отец,cheap,0,поместится,влезет
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",13,1,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,тёмный рыцарь,expensive,1,поместится,влезет
3,4,крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",15,2,крёстный отец 2,крёстный отец 2,крёстный отец 2,крёстный отец 2,expensive,0,не поместится,не влезет
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",22,2,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,12 разгневанных мужчин,expensive,0,не поместится,не влезет
5,6,список шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",15,1,список шиндлера,список шиндлера,список шиндлера,список шиндлера,cheap,0,не поместится,не влезет
6,7,властелин колец: возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",35,3,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,властелин колец: возвращение короля,expensive,1,не поместится,не влезет
7,8,криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",18,1,криминальное чтиво,криминальное чтиво,криминальное чтиво,криминальное чтиво,expensive,0,не поместится,не влезет


## 3 *. Найти потеряшку

От начальства поступила задача найти конкретного клиента и всю информацию по нему. Однако никто не помнит первый символ в его  `client_id`. Известно, что заканчивается на `G35`.<br><br>

Найдите этого клиента и выведите всю информацию по нему (все строки датафрейма).

*Возможный вариант выполнения:*<br>
*1. Применить связку **apply + lambda** к столбцу `client_id`, вернув срезом только 3 последних символа*<br>
*2. С помощью получившегося **series** с урезанной версией  `client_id` создать логическое выражение (проверить на равенство известной части айдишника)*<br>
*3. С помощью получившегося логического выражения отфильтровать датайрефм, применив логическую индексацию.*<br>

In [186]:
# your cod
payments['client_id_short'] = payments['client_id'].apply(lambda str: str[-3:]) 
payments[payments['client_id_short'] == "G35"]

,pay_date,film_id,part_of_day,client_id,count_tickets,client_id_short
51,2024-02-07,6,утренний,JG35,2.0,G35
71,2024-04-03,20,утренний,JG35,5.0,G35
342,2024-03-08,17,дневной,JG35,2.0,G35
411,2024-03-16,9,дневной,JG35,3.0,G35
564,2024-01-06,25,дневной,JG35,4.0,G35
628,2024-04-16,4,вечерний,JG35,5.0,G35
833,2024-04-06,39,дневной,JG35,4.0,G35
998,2024-02-05,35,утренний,JG35,5.0,G35
1087,2024-02-27,23,утренний,JG35,4.0,G35
1757,2024-01-15,42,вечерний,JG35,3.0,G35


## 4. Создайте столбец "Детективчик"

В датафрейме **films** на основе столбца `kind_film` создайте новый столбец с названием "Детективчик". Фильм "Детективчик", если в перечислении жанров в `kind_film` встречается "детектив"

In [192]:
# your code 
films['kind_film_list'] = films['kind_film'].to_list()

def detectivchik(kind_film_list):
    if 'детектив' in kind_film_list:
        return 'Детективчик'
    else:
        return '-'

films ['Детективчик'] = films['kind_film_list'].apply(detectivchik)
films


,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,kind_film_list,Детективчик
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,драма,-
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма","детектив, драма",Детективчик
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма","боевик, детектив, драма",Детективчик
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма","детектив, драма",Детективчик
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма","детектив, драма",Детективчик
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм","байопик, драма, исторический фильм",-
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма","боевик, приключение, драма",-
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма","детектив, драма",Детективчик
8,9,Властелин колец: Братство Кольца,290,2001,Питер Джексон,"боевик, приключение, драма","боевик, приключение, драма",-
9,10,"Хороший, плохой, злой",450,1966,Серджо Леоне,"приключение, вестерн","приключение, вестерн",-


## 5. Что-то такое было на Семинаре 1...

В датафрейме **films** cоздайте столбец при помощи np.where, где проставляется 1, если удовлетворяет условию, и 0, если нет.

**Условие**: все фильмы с ценой либо меньше 190, либо больше 300. Которые снял НЕ Кристофер Нолан в датафрейме `films`

Посчитайте долю фильмов с таким условием

In [199]:
# your code 

films['условие'] = np.where(
    (films['price_ticket'] < 190) |
    (films['price_ticket'] > 300) &
    (films['filmmaker'] != 'Кристофер Нолан'),
    1, 
    0
)
mean_films = round(films['условие'].mean(), 2)
print(f'Доля фильмов с ценой билета менее 190 руб. и более 300 руб., которые не снимал Кристофер Нолан, составляет {mean_films}')

Доля фильмов с ценой билета менее 190 руб. и более 300 руб., которые не снимал Кристофер Нолан, составляет 0.31
